# MMClassification tools tutorial on Colab

In this tutorial, we will introduce the following content:

* How to install MMClassification
* Prepare data
* Prepare the config file
* Train and test model with shell command

## Install MMClassification

Before using MMClassification, we need to prepare the environment with the following steps:

1. Install Python, CUDA, C/C++ compiler and git
2. Install PyTorch (CUDA version)
3. Install mmcv
4. Clone mmcls source code from GitHub and install it

Because this tutorial is on Google Colab, and the basic environment has been completed, we can skip the first two steps.

### Check environment

In [1]:
!pwd

/mnt/storage/Projects/tubetech


In [ ]:
# Check nvcc version
!nvcc -V

In [ ]:
# Check GCC version
!pip install torch, torchvision

In [ ]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

### Clone and install MMClassification

Now we clone the latest mmcls repository from GitHub and install it by the `mim` tool.

> *mim is a light-weight command-line tool to setup appropriate environment for OpenMMLab repositories according to PyTorch and CUDA version. It also has some useful functions for deep-learning experiments.*

In [ ]:
# Clone mmcls repository and checkout to the 1.x branch
!git clone -b 1.x https://github.com/open-mmlab/mmclassification.git
%cd mmclassification/

# Install MMClassification from source by mim
!pip install openmim
!mim install -e .

In [2]:
%cd mmclassification/

/mnt/storage/Projects/tubetech/mmclassification


In [3]:
# Check MMClassification installation
import mmcls
print(mmcls.__version__)

1.0.0rc6


## Prepare data

In [ ]:
!curl -L "https://universe.roboflow.com/ds/7AnNVv41OV?key=Tp2iAqJywA" > roboflow.zip;
!mkdir -p data/rust_dataset
!unzip -q roboflow.zip -d ./data/rust_dataset
!rm roboflow.zip

In [ ]:
# Pick an image and visualize it
from PIL import Image
Image.open('data/rust_dataset/test/Slightly Visible Rust/Steel-145-_jpg.rf.99e877b3d3e2e37dd09a4e1260378e7b.jpg')

### Support new dataset

We have two methods to support a new dataset in MMClassification.

The simplest method is to re-organize the new dataset as the format of a dataset supported officially (like `CustomDataset`). And you can also create a new dataset class. More details are in [the docs](https://mmclassification.readthedocs.io/en/dev-1.x/user_guides/dataset_prepare.html).

In this tutorial, for convenience, we have re-organized the cats & dogs dataset as the format of `CustomDataset`.

Besides image files, it also includes the training/validation/test annotation files. And every line includes an file path and the corresponding label.

```
...
cats/cat.3769.jpg 0
cats/cat.882.jpg 0
dogs/dog.3881.jpg 1
dogs/dog.3377.jpg 1
...
```

## Train and test model with shell commands

You can use shell commands provided by MMClassification to do the following task:

1. Train a model
2. Fine-tune a model
3. Test a model
4. Inference with a model

The procedure to train and fine-tune a model is almost the same. And we have introduced how to do these tasks with Python API. In the following, we will introduce how to do them with shell commands. More details are in [the docs](https://mmclassification.readthedocs.io/en/dev-1.x/user_guides/train_test.html).

### Fine-tune a model

The steps to fine-tune a model are as below:

1. Prepare the custom dataset.
2. Create a new config file of the task.
3. Start training task by shell commands.

We have finished the first step, and then we will introduce the next two steps.


#### Create a new config file

To reuse the common parts of different config files, we support inheriting multiple base config files. For example, to fine-tune a MobileNetV2 model, the new config file can create the model's basic structure by inheriting `configs/_base_/models/mobilenet_v2_1x.py`.

According to the common practice, we usually split whole configs into four parts: model, dataset, learning rate schedule, and runtime. Configs of each part are saved into one file in the `configs/_base_` folder.

And then, when creating a new config file, we can select some parts to inherit and only override some different configs.

The head of the final config file should look like:

```python
_base_ = [
    '../_base_/models/mobilenet_v2_1x.py',
    '../_base_/schedules/imagenet_bs256_epochstep.py',
    '../_base_/default_runtime.py'
]
```

Here, because the dataset configs are almost brand new, we don't need to inherit any dataset config file.

Of course, you can also create an entire config file without inheritance, like `configs/lenet/lenet5_mnist.py`.

After that, we only need to set the part of configs we want to modify, because the inherited configs will be merged to the final configs.

#### Use shell command to start fine-tuning

We use `tools/train.py` to fine-tune a model:

```shell
python tools/train.py ${CONFIG_FILE} [optional arguments]
```

And if you want to specify another folder to save log files and checkpoints, use the argument `--work_dir ${YOUR_WORK_DIR}`.

Here we use the `MobileNetV2` model and cats & dogs dataset as an example:


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
CONFIG = 'configs/deit3/deit3-small-p16_64xb64_in1k-384px_rust.py'
MODEL_NAME = 'deit3-small-p16_64xb64_in1k-384px_rust'
WORK_DIR = "/content/gdrive/MyDrive/tubetech_drone_inspection"

In [ ]:
!python tools/train.py \
  {CONFIG} \
  --work-dir {WORK_DIR}/{MODEL_NAME}_640

### save the model

In [ ]:
!cp "$(cat work_dirs/{MODEL_NAME}/last_checkpoint)" /content/gdrive/MyDrive/tubetech_drone_inspection/rust_transformer.pth

### Test a model

We use `tools/test.py` to test a model:

```
python tools/test.py ${CONFIG_FILE} ${CHECKPOINT_FILE} [optional arguments]
```

Here are some optional arguments:

- `--out`: Output the metric result to the specified file.
- `--dump`: Dump the prediction output of every sample to the specified file.

More details are in the help docs of `tools/test.py`.

Here we still use the `MobileNetV2` model we fine-tuned.

In [2]:
# weights = !"$(cat work_dirs/{MODEL_NAME}/last_checkpoint)"
WEIGHTS = '/home/innovation/Projects/tubetech/mmclassification/work_dirs/deit3-large-p16_64xb16_boiler_defects-640px/best_accuracy_top1_epoch_13.pth'

In [3]:
CONFIG = 'configs/deit3/deit3-large-p16_64xb16_boiler_defects-640px.py'

In [8]:
!python tools/test.py {CONFIG} {WEIGHTS} --out-item pred --out result.pkl

06/23 16:50:38 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.0 (default, Mar  3 2022, 09:58:08) [GCC 7.5.0]
    CUDA available: True
    numpy_random_seed: 0
    GPU 0,1: GeForce GTX 1080 Ti
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 10.2, V10.2.8
    GCC: gcc (Ubuntu 6.5.0-2ubuntu1~18.04) 6.5.0 20181026
    PyTorch: 2.0.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code

In [ ]:
import mmengine

results = mmengine.load("result.pkl")
# Output the first samples' ground truth and prediction.
print('Ground truth:', results[0]['gt_label'])
print('Prediction:', results[0]['pred_label'])

In [ ]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard
import tensorflow as tf

In [ ]:
%tensorboard --logdir {WORK_DIR}

### Inference with a model

Sometimes we want to save the inference results on a image, just use the command below.

```shell
python demo/image_demo.py ${IMAGE_FILE} ${CONFIG_FILE} ${CHECKPOINT_FILE}
```


In [ ]:
!python demo/image_demo.py --checkpoint {WEIGHTS} --show  --show-dir work_dirs/deit3-small-p16_64xb64_in1k-384px_rust/output 'work_dirs/boiler_square.jpg'  configs/deit3/deit3-small-p16_64xb64_in1k-384px_rust.py

In [ ]:
!python tools/visualizations/browse_dataset.py configs/mobilenet_v2/mobilenet_v2_1x_rust.py -n 10 -o tmp -m pipeline


In [1]:
!pip install grad-cam>=1.3.6

In [14]:
PIC = '/home/innovation/Projects/tubetech/boiler_defects_dataset2/train/defect/2633_L76_jpg.rf.654226d3abb7bdbe260c4d407c270384.jpg'
!python tools/visualizations/vis_cam.py \
     {PIC} \
    {CONFIG} \
    {WEIGHTS} \
    --method EigenCAM \
    --save-path layer_activations/boiler_square.jpg \
    --vit-like \
   --num-extra-tokens 21

!cp {PIC} 'layer_activations/original.jpg'

06/23 17:06:18 - mmengine - INFO - Because batch augmentations are enabled, the data preprocessor automatically enables the `to_onehot` option to generate one-hot format labels.
Loads checkpoint by local backend from path: /home/innovation/Projects/tubetech/mmclassification/work_dirs/deit3-large-p16_64xb16_boiler_defects-640px/best_accuracy_top1_epoch_13.pth
Automatically choose the last norm layer before the final attention block as target_layer..


In [ ]:
!pip install fvcore

In [ ]:
!python tools/analysis_tools/get_flops.py {CONFIG} --shape 640